In [ ]:
import os
import cv2
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopandas import GeoSeries
from shapely.geometry import Polygon
from rasterio.windows import Window
from rasterio.plot import reshape_as_image

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, Flatten, Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
%matplotlib inline

In [ ]:
# read the labels dataframe
farmpin_data = pd.read_csv("/home/ymi/data/ucu_data/Farmpin_training.csv")

In [ ]:
# find all images
dataset = os.listdir("/home/ymi/data/ucu_data/images_cropped_rgb")
farmIds = [int(img_name.split(".")[0]) for img_name in dataset]

In [ ]:
# prepare the data
for num, row in farmpin_data.iterrows():
    img_path = os.path.join("/home/ymi/data/ucu_data/images_cropped_rgb", str(row['field_id']) + ".png")
    img = cv2.imread(img_path)
    if isinstance(img, np.ndarray):
        farmpin_data.loc[num, 'img_path'] = img_path
        farmpin_data.loc[num, 'size'] = img.shape[0] * img.shape[1]

In [ ]:
# filter fields with area higher than 15 pixels 

farmpin_data.dropna(inplace=True)

farmpin_data['size'].min()

farmpin_data = farmpin_data[farmpin_data['size'] >= 15]

In [ ]:
# prepare y labels
farmpin_data['crop_id'] = farmpin_data['crop_id'] - 1

y = to_categorical(farmpin_data['crop_id'].tolist(), num_classes = 9)

In [ ]:
def load_image(img_path):
      
    img = cv2.imread(img_path)

    return img

In [ ]:
def crop_image(image, w, h):
    center = np.array(image.shape) / 2
    x = int(center[1] - w/2)
    y = int(center[0] - h/2)
    crop_img = image[int(y):int(y+h), int(x):int(x+w), :]
    return crop_img

In [ ]:
#generate X
def generate_X(img_list, w, h):
    input_size = int(w * h * 3)
    X = np.empty((len(img_list), input_size))

    for i, item in enumerate(img_list):
        img = load_image(item)
        # print(img.shape)
        if img.shape[0] > h and img.shape[1] > h:
            img = crop_image(img, w, h)
        img = img.flatten()
        if img.shape[0] > input_size:
            img = img[:input_size]
        X[i,] = img
    
    # normalize 
    X = X / 127.5
    X -= 1.0
    return X


In [ ]:
X_train = generate_X(farmpin_data['img_path'], 4, 4)
X_train.shape

In [ ]:
y.shape

In [ ]:
def build_model_dense(input_shape, num_classes):
    inputs = Input(input_shape)
    
    # 32 layers with sigmoid activation
    # 16 layers
    # output with softmax activation
    hidden_1 = Dense(32, activation = 'relu')(inputs)
    hidden_2 = Dense(16, activation = 'relu')(hidden_1)
    outputs = Dense(num_classes, activation = 'softmax')(hidden_2)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    
    
    return model


In [ ]:
# create model with input 48 and output 9



In [ ]:
# check model summary 

In [ ]:

model_checkpoint = ModelCheckpoint("model_class.hdf5", save_best_only=True, verbose=1, monitor='accuracy', mode='max')

In [ ]:
# compile model with tf.keras.optimizers.Adam(0.001), loss tf.keras.losses.CategoricalCrossentropy(from_logits=False), tf.keras.metrics.CategoricalAccuracy()]

In [ ]:
# fit model
classification_model.fit(X_train,
          y,
          batch_size=256, 
          epochs=1000,
          verbose=1,
          callbacks=[model_checkpoint])

In [ ]:
# predict X_train


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns

In [ ]:
conf_matrix = confusion_matrix(farmpin_data['crop_id'].tolist(), np.argmax(y_pred, axis=1))
sns.heatmap(conf_matrix, annot=True)

In [ ]:
# task create train, val, test split 
# train model with X_train, y_train, X_val, y_val
# and predict on X_test, y_test